# CNN models - model training using transfer learning



## Python imports

In [1]:
from keras.preprocessing.image import ImageDataGenerator
import pandas as pd
import sklearn.model_selection
import tensorflow as tf
import tensorflow_hub as hub
import time

2025-06-18 09:08:01.873006: I tensorflow/tsl/cuda/cudart_stub.cc:28] Could not find cuda drivers on your machine, GPU will not be used.
2025-06-18 09:08:02.081785: I tensorflow/tsl/cuda/cudart_stub.cc:28] Could not find cuda drivers on your machine, GPU will not be used.
2025-06-18 09:08:02.083184: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2025-06-18 09:08:02.977238: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT


## Configuration

In [2]:
import ipywidgets as widgets

# MODEL_CHARACTERISTICS
MODELS = {
    "InceptionV1" : {
        "url": "https://www.kaggle.com/models/google/inception-v1/TensorFlow2/feature-vector/2",
        "dim": 224
    },
    "InceptionV2" : {
        "url": "https://www.kaggle.com/models/google/inception-v2/TensorFlow2/feature-vector/2",
        "dim": 224
    },
    "MobileNetV1" : {
        "url": "https://www.kaggle.com/models/google/mobilenet-v1/TensorFlow2/100-224-feature-vector/2",
        "dim": 224
    },
    "MobileNetV2" : {
        "url": "https://www.kaggle.com/models/google/mobilenet-v2/TensorFlow2/100-224-feature-vector/2",
        "dim": 224
    },
    "MobileNetV3" : {
        "url": "https://www.kaggle.com/models/google/mobilenet-v3/TensorFlow2/small-100-224-feature-vector/1",
        "dim": 224
    },
    "NasNetMobile" : {
        "url": "https://www.kaggle.com/models/google/nasnet/TensorFlow2/mobile-feature-vector/2",
        "dim": 224
    },
    "ResNetV1" : {
        "url": "https://www.kaggle.com/models/google/resnet-v1/TensorFlow2/50-feature-vector/2",
        "dim": 224
    },
    "ResNetV2" : {
        "url": "https://www.kaggle.com/models/google/resnet-v2/TensorFlow2/50-feature-vector/2",
        "dim": 224
    },
}
P_DIM=224
NUM_CLASSES=21
CT_CFG = widgets.FloatSlider(description="Convergenge threshold:", value=1.0, min=0.25, max=1.00, step=0.01)
DROPOUT_CFG = widgets.FloatSlider(description="Dropout (0: no dropout):", value=0.2, min=0.0, max=0.5, step=0.1)
MAX_STEPS_CFG = widgets.IntSlider(description="Max. training steps:", value=25, min=1, max=100, step=10)
BATCH_SIZE_CFG = widgets.IntSlider(description="Batch size:", value=32, min=32, max=500)
display(widgets.VBox([CT_CFG, DROPOUT_CFG, MAX_STEPS_CFG, BATCH_SIZE_CFG]))

Widget Javascript not detected.  It may not be installed or enabled properly. Reconnecting the current kernel may help.


In [3]:

global P_CT
P_CT = CT_CFG.value
P_DROPOUT = DROPOUT_CFG.value
P_MAX_STEPS = MAX_STEPS_CFG.value
P_BATCH_SIZE = BATCH_SIZE_CFG.value
print('','Input dim:', P_DIM, '\n',
      'Num classes', NUM_CLASSES, '\n',
      'Conv. thr.:', P_CT, '\n',
      'Dropout:', P_DROPOUT, '\n',
      'Max steps:', P_MAX_STEPS, '\n',
      'Batch size:', P_BATCH_SIZE)

 Input dim: 224 
 Num classes 21 
 Conv. thr.: 1.0 
 Dropout: 0.2 
 Max steps: 25 
 Batch size: 32


## Read train data set

In [12]:
train_df = pd.read_csv('datasets/vision/train.tsv', sep='\t')
SOURCE_FILTER = 'Redmi'
if SOURCE_FILTER != None:
    train_df = train_df[train_df.source == SOURCE_FILTER]
train_df

,label,set,time,source,image,video
2059,SS,train,4,Redmi,video_data/SS/train_Xiaomi_0004_0121.jpeg,../dados_indoor_location/Xiaomi/Vision/1653909...
2060,SS,train,5,Redmi,video_data/SS/train_Xiaomi_0005_0151.jpeg,../dados_indoor_location/Xiaomi/Vision/1653909...
2061,SS,train,6,Redmi,video_data/SS/train_Xiaomi_0006_0181.jpeg,../dados_indoor_location/Xiaomi/Vision/1653909...
2062,SS,train,7,Redmi,video_data/SS/train_Xiaomi_0007_0211.jpeg,../dados_indoor_location/Xiaomi/Vision/1653909...
2063,SS,train,8,Redmi,video_data/SS/train_Xiaomi_0008_0241.jpeg,../dados_indoor_location/Xiaomi/Vision/1653909...
...,...,...,...,...,...,...
3738,AT_I2,train,106,Redmi,video_data/AT_I2/train_Xiaomi_0106_3181.jpeg,../dados_indoor_location/Xiaomi/Vision/1653920...
3739,AT_I2,train,107,Redmi,video_data/AT_I2/train_Xiaomi_0107_3211.jpeg,../dados_indoor_location/Xiaomi/Vision/1653920...
3740,AT_I2,train,108,Redmi,video_data/AT_I2/train_Xiaomi_0108_3241.jpeg,../dados_indoor_location/Xiaomi/Vision/1653920...
3741,AT_I2,train,109,Redmi,video_data/AT_I2/train_Xiaomi_0109_3271.jpeg,../dados_indoor_location/Xiaomi/Vision/1653920...


## Define image data generator for train dataset 

In [13]:
datagen = ImageDataGenerator(rescale=1.0/255.0)

train_dataset = datagen.flow_from_dataframe(
    train_df,
    target_size=(P_DIM,P_DIM),
    batch_size=P_BATCH_SIZE,
    class_mode='sparse',
    shuffle=True,
    seed=1234567,
    x_col='image',
    y_col='label'
)

Found 1684 validated image filenames belonging to 21 classes.



## Train models

In [14]:
class CustomCallback(tf.keras.callbacks.Callback):
    def on_epoch_end(self, epoch, logs=None):
        global P_CT
        if logs.get('accuracy') >= P_CT:
            self.model.stop_training = True

MODEL_FILTER = None

for model_id in MODELS:
    if MODEL_FILTER != None and MODEL_FILTER not in model_id:
        continue
    print('==> Deriving model', model_id)
    kl = hub.KerasLayer('models/vision/transfer_learning/' + model_id, trainable=False, input_shape=(P_DIM, P_DIM, 3))
    fl = tf.keras.layers.Dense(NUM_CLASSES, activation='softmax')
    if P_DROPOUT > 0.0:
        model = tf.keras.Sequential([ kl, tf.keras.layers.Dropout(P_DROPOUT), fl])
    else:
        model = tf.keras.Sequential([ kl, fl])
    model.summary()
    model.compile(optimizer='adam',
                  loss=tf.keras.losses.SparseCategoricalCrossentropy(),
                  metrics=['accuracy'])
    t_start = time.time()
    history = model.fit(train_dataset, epochs=P_MAX_STEPS, callbacks = [CustomCallback()])
    t_end = time.time()
    print('Training time', t_end - t_start)
    if SOURCE_FILTER != None:
        model_id = model_id + '_' + SOURCE_FILTER
    model.save('models/vision/' + model_id)
    converter = tf.lite.TFLiteConverter.from_saved_model('models/vision/' + model_id) # path to the SavedModel directory
    tflite_model = converter.convert()
    with open('models/vision/' + model_id + '/model.tflite', 'wb') as f:
      f.write(tflite_model)

==> Deriving model MobileNetV1
Model: "sequential_2"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 keras_layer_2 (KerasLayer)  (None, 1024)              3228864   
                                                                 
 dropout_2 (Dropout)         (None, 1024)              0         
                                                                 
 dense_2 (Dense)             (None, 21)                21525     
                                                                 
Total params: 3250389 (12.40 MB)
Trainable params: 21525 (84.08 KB)
Non-trainable params: 3228864 (12.32 MB)
_________________________________________________________________
Epoch 1/25
53/53 [==============================] - 16s 268ms/step - loss: 2.3161 - accuracy: 0.3444
Epoch 2/25
53/53 [==============================] - 14s 253ms/step - loss: 1.1428 - accuracy: 0.6710
Epoch 3/25
53/53 [========================

INFO:tensorflow:Assets written to: models/vision/MobileNetV1_Redmi/assets
2025-06-18 09:25:20.031476: W tensorflow/compiler/mlir/lite/python/tf_tfl_flatbuffer_helpers.cc:364] Ignored output_format.
2025-06-18 09:25:20.031506: W tensorflow/compiler/mlir/lite/python/tf_tfl_flatbuffer_helpers.cc:367] Ignored drop_control_dependency.
2025-06-18 09:25:20.031717: I tensorflow/cc/saved_model/reader.cc:45] Reading SavedModel from: models/vision/MobileNetV1_Redmi
2025-06-18 09:25:20.040571: I tensorflow/cc/saved_model/reader.cc:91] Reading meta graph with tags { serve }
2025-06-18 09:25:20.040600: I tensorflow/cc/saved_model/reader.cc:132] Reading SavedModel debug info (if present) from: models/vision/MobileNetV1_Redmi
2025-06-18 09:25:20.065539: I tensorflow/cc/saved_model/loader.cc:231] Restoring SavedModel bundle.
2025-06-18 09:25:20.257797: I tensorflow/cc/saved_model/loader.cc:215] Running initialization op on SavedModel bundle at path: models/vision/MobileNetV1_Redmi
2025-06-18 09:25:20.3